# PySCF tutorial

Here we give a small tutorial on how to use pySCF. Please refer to the online pySCF documentation for a full reference.

Note that you need to install pySCF on your machine in order to be able to run the code below.


In [ ]:
# %load qat/dqs/qchem/pyscf_tools.py
from functools import reduce
import numpy as np

from pyscf import gto, scf, fci, mp, ci
from pyscf import ao2mo

def compute_integrals(molecule, mo_coeff, hcore):
    """
    for a given molecule, compute 1-body and 2-body integrals
    """
    ### no spin dof
    one_electron_compressed = reduce(np.dot, (mo_coeff.T, hcore, mo_coeff))
    n_orbitals = mo_coeff.shape[1]
    one_electron_integrals = one_electron_compressed.reshape(n_orbitals, n_orbitals).astype(float)
    two_electron_compressed = ao2mo.kernel(molecule, mo_coeff)
    #1: no permutation symmetry
    two_electron_integrals = ao2mo.restore(1, two_electron_compressed, n_orbitals)
    two_electron_integrals = np.asarray(two_electron_integrals.transpose(0, 2, 3, 1), order='C')
    return one_electron_integrals, two_electron_integrals


def perform_pyscf_computation(geometry, basis, spin, charge, verbose=False):
    #define molecule in pySCF format
    molecule = gto.Mole()
    molecule.atom = geometry
    molecule.basis = basis
    molecule.spin = spin
    molecule.charge = charge
    molecule.symmetry = False
    molecule.build()

    # Run SCF.
    scf_worker = scf.ROHF(molecule) if molecule.spin else scf.RHF(molecule)
    scf_worker.verbose = 0
    scf_worker.run()
    hf_energy = float(scf_worker.e_tot)

    if verbose:
        print("HF energy=", hf_energy)

    one_body_integrals, two_body_integrals = compute_integrals(molecule,
                                                               scf_worker.mo_coeff,
                                                               scf_worker.get_hcore())

    nuclear_repulsion = float(molecule.energy_nuc())
    orbital_energies = scf_worker.mo_energy.astype(float)
    nels = molecule.nelectron

    # Run CISD
    cisd = ci.CISD(scf_worker)
    cisd.verbose = 0
    cisd.run()
    rdm1 = cisd.make_rdm1()

    # Run MP2.
    #note: molecule.spin must be 0
    mp2 = mp.MP2(scf_worker)
    mp2.verbose = 0
    mp2.run()
    mp2_energy = scf_worker.e_tot + mp2.e_corr

    if verbose:
        print("MP2 energy=", mp2_energy)

    # Run FCI
    fci_worker = fci.FCI(molecule, scf_worker.mo_coeff)
    fci_worker.verbose = 0
    fci_energy = fci_worker.kernel()[0]

    if verbose:
        print("FCI energy=", fci_energy)


    return rdm1, orbital_energies, nuclear_repulsion, nels, one_body_integrals,\
           two_body_integrals


We provide a simple wrapper that carries out these steps in ``qat.dqs.qchem.pyscf_tools``: 

In [ ]:
import numpy as np
from qat.dqs.qchem.pyscf_tools import perform_pyscf_computation
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., 0.7414))]
basis = "sto-3g"
#geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.75))]
#basis = "6-31g"

rdm1, orbital_energies, nuclear_repulsion,\
nels, one_body_integrals, two_body_integrals, info = perform_pyscf_computation(geometry=geometry, basis=basis, spin=0,
                                                                               charge=0, verbose=True)

These outputs can be used as a starting point for a VQE computation, see [VQE on a H2 molecule](./vqe_ucc_h2.ipynb).